In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/dataset-tkc/

/content/gdrive/MyDrive/dataset-tkc


In [3]:
!ls

best_model_age_tunning.h5     dataset-adience
best_model_gender_tunning.h5  kaggle.json


In [4]:
!mkdir -p ~/.kaggle #membuat directory bernama kaggle 
!cp kaggle.json ~/.kaggle/ #copy file kaggle.json upload tadi ke directory baru
!chmod 600 ~/.kaggle/kaggle.json #berikan izin untuk file kaggle.jsonv

In [5]:
%cd /content/gdrive/MyDrive/dataset-tkc/dataset-adience

/content/gdrive/MyDrive/dataset-tkc/dataset-adience


In [ ]:
!kaggle datasets download -d arcarcarc/adience-dataset-preprocessed

adience-dataset-preprocessed.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
from zipfile import ZipFile
file_name = "/content/gdrive/MyDrive/dataset-tkc/dataset-adience/adience-dataset-preprocessed.zip"

with ZipFile(file_name,'r') as zip :
  zip.extractall()
  print('Berhasil')

Berhasil


In [8]:
!ls

adience-dataset-preprocessed.zip  datasets


In [10]:
!pip install streamlit

     |████████████████████████████████| 9.1 MB 23.0 MB/s 
     |████████████████████████████████| 4.3 MB 29.0 MB/s 
     |████████████████████████████████| 76 kB 4.1 MB/s 
     |████████████████████████████████| 178 kB 45.0 MB/s 
     |████████████████████████████████| 111 kB 50.8 MB/s 
     |████████████████████████████████| 180 kB 59.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 126 kB 58.8 MB/s 
     |████████████████████████████████| 791 kB 40.7 MB/s 
     |████████████████████████████████| 374 kB 50.2 MB/s 
  Created wheel for pympler: filename=Pympler-0.9-py3-none-any.whl size=164824 sha256=75dc02577f0621bec3008f3726b1421c6bd8f9300dae202bb3de5bb52e5e9b90
  Stored in directory: /root/.cache/pip/wheels/1a/f3/d8/35d5614ea4ddd295ffb9372a5f2f9570d9593d1ea4be33ec6d
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=4576cfde0d1dba25dbd6cbac894f43fcb05a7a1a04f3a261f59ea1a242c2dbb3
  Stored in dire

In [11]:
!pip -q install pyngrok==4.1.1

In [55]:
%%writefile app.py

import os
import cv2
import numpy as np
import streamlit as st
import tensorflow as tf
import random
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input as mobilenet_v2_preprocess_input
import matplotlib.pyplot as plt
import matplotlib.image as mping

model_gender = tf.keras.models.load_model("/content/gdrive/MyDrive/dataset-tkc/best_model_gender_tunning.h5")

map_dict_gender = {
    0 : 'female',
    1 : 'male'
}

def retrieval_female():
  base_female = os.path.join("/content/gdrive/MyDrive/dataset-tkc/dataset-adience/datasets/gender/female")
  total_female = os.listdir(base_female)
  female_list = []

  fname1 = [os.path.join(base_female,fname)
        for fname in total_female[:15]]

  random_list=[]
  for _ in range(5):
      sample_num = random.randint(0, 14)#random pilih gambar
      # im_name = total_female.iloc[sample_num].original_image#pilih gambar dengan fungsi random
      im_path = fname1[sample_num] 
      image = cv2.imread(im_path)
      random_list.append(fname1[sample_num])
      female_list.append(image)
  return female_list, random_list[:6]

def retrieval_male():
  base_male = os.path.join("/content/gdrive/MyDrive/dataset-tkc/dataset-adience/datasets/gender/male")
  total_male = os.listdir(base_male)
  male_list = []

  fname2 = [os.path.join(base_male,fname)
        for fname in total_male[:15]]

  random_list=[]
  for _ in range(5):
      sample_num = random.randint(0, 14)#random pilih gambar
      # im_name = total_female.iloc[sample_num].original_image#pilih gambar dengan fungsi random
      im_path = fname2[sample_num] 
      image = cv2.imread(im_path)
      random_list.append(fname2[sample_num])
      male_list.append(image)
  return male_list, random_list[:6]

def build_figure_female(fname1) :
  ncolu = 5 #inisialisasi var
  nrow = 1 #inisialisasi var
  fig = plt.gcf() #mengatur figure/canvas
  # fig.set_size_inches(ncolu,nrow*5) #mengatur ukuran canvas
  for i,fname in enumerate(fname1): #perulangan, enumerate berfungsi untuk memberikan nilai angka pada tiap2 file yang dipanggil --> (0, gmbr benign), (1, gmbr malignant)
      if i+1 == 6:
        break
      sp = plt.subplot(1,5,i+1) #berfungsi untuk peletakan file di dalam canvas
      sp.axis('off') #gak ada penggaris disamping foto
      img = mping.imread(fname) #membaca gambar sebagai array
      plt.imshow(img)
  st.pyplot(fig)


def build_figure_male(fname2) :
  ncolu = 5 #inisialisasi var
  nrow = 1 #inisialisasi var
  fig = plt.gcf() #mengatur figure/canvas
  for i,fname in enumerate(fname2): #perulangan, enumerate berfungsi untuk memberikan nilai angka pada tiap2 file yang dipanggil --> (0, gmbr benign), (1, gmbr malignant)
      if i+1 == 6:
        break
      sp = plt.subplot(1,5,i+1) #berfungsi untuk peletakan file di dalam canvas
      sp.axis('off') #gak ada penggaris disamping foto
      img = mping.imread(fname) #membaca gambar sebagai array
      plt.imshow(img)
  st.pyplot(fig)


def processing(uploaded_file):
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    opencv_image = cv2.imdecode(file_bytes, 1)
    opencv_image = cv2.cvtColor(opencv_image, cv2.COLOR_BGR2RGB)
    opencv_image = cv2.resize(opencv_image,(227,227))
    # Now do something with the image! For example, let's display it:
    st.image(opencv_image, channels="RGB")
    # data = np.asarray(opencv_image)
    test = np.asarray(opencv_image)
    img_reshape = test[np.newaxis,...]
    return img_reshape

def predict_gender(model_gender, img):
    # resized = mobilenet_v2_preprocess_input(resized)
    prediction_gender = model_gender.predict(img).argmax()
    return prediction_gender

### load file
st.text('masukan gambar')
uploaded_file = st.file_uploader("Choose a image file", type="jpg")
if uploaded_file is not None:
    img = processing(uploaded_file)
    gender = predict_gender(model_gender,img)

    Genrate_pred = st.button("Generate Prediction")    
    if Genrate_pred:
      if gender == 0: 
        st.title("Predicted Label for the image is {}".format(map_dict_gender[gender]))
        female_list, fname1 = retrieval_female()
        build_figure_female(fname1)
      else:
        st.title("Predicted Label for the image is {}".format(map_dict_gender[gender]))
        male_list, fname2 = retrieval_male()
        build_figure_male(fname2)


Overwriting app.py


In [12]:
from pyngrok import ngrok

In [13]:
#Akun Randy 
!ngrok authtoken 21u6r0Z57gwNHC04qVGUYezNsyV_5vPevfFFgibP4YCDMF8aU

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [14]:
!streamlit run app.py &>/dev/null&

In [15]:
publ_url = ngrok.connect(port='8501')

In [16]:
publ_url 

'http://7b24-35-245-121-157.ngrok.io'